In [1]:
import requests
from pprint import pprint
import re

In [2]:
# init session 
S = requests.Session()

# set URL
URL = "https://en.wikipedia.org/w/api.php"

In [3]:
# get sections
PARAMS = {
    "action": "parse",
    "pageid": "64618",
    "format": "json",
    "prop": "sections"
}

sectionsR = S.get(url=URL, params=PARAMS)
sectionDATA = sectionsR.json()



In [4]:
# check sections
# pprint(sectionDATA["parse"]["sections"])

In [5]:
sections = [sec['anchor'] for sec in sectionDATA["parse"]["sections"]]

In [6]:
# create section list filtering out references section
sec_list = [sec for sec in sectionDATA["parse"]["sections"] if not sec['anchor'] == 'References']
len(sec_list)

73

In [24]:
# create nested poets dictionary from section list based on toclevel
# adding 'Z' section by hand consider refactor
poets = {}
is_new_parent = True
parent = ''
for i in range(len(sec_list) - 1):
    if sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 1:
        poets[sec_list[i]['anchor']] = []
    elif sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 2:
        poets[sec_list[i]['anchor']] = {}
        parent = sec_list[i]['anchor']
    elif sec_list[i]['toclevel'] == 2:
        poets[parent][sec_list[i]['anchor']] = []
poets['Z'] = []
#poets

In [8]:
secs = [sec for sec, val in poets.items()]


In [9]:
# get poet data
PARAMS = {
    "action": "parse",
    "pageid": "64618",
    "format": "json",
    "prop": "wikitext"
}

textR = S.get(url=URL, params=PARAMS)
textDATA = textR.json()



In [10]:
# visualize poet data
#pprint(textDATA["parse"]["wikitext"]["*"].split('\n')[4:10])

In [25]:
# parse poet data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')[3:-10] # [::-1]
this_sec = {}
sec = ''
sub_sec = ''
with open('error_lines.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines):
        if index == 23:
            print(line)
        if re.match('==\w', line[0:3]):
            sec = line.strip().replace("=", "")
            if len(this_sec) > 0:
                poets[secs[secs.index(sec) - 1]] = this_sec
            this_sec = {}
        if type(poets[sec]).__name__ == 'list':
            if re.match('\*\[\[', line[0:3]):
                try:
                    poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                    #print(poet)

                except:
                    print("##########################")
                    print("#### poet parse error ####")
                    print(poet_method, index, line)
                    filepath.write(line + '\n')
                    print(line[-2:])
                    print("##########################")

                date_m = re.search('\((.*)\)', line)

                try:
                    date = date_m.group(0)
                except:
                    print("##########################")
                    print("#### date parse error ####")
                    print(index, line)
                    filepath.write(line + '\n')
                    date = ""
                    print("##########################")
                try:
                    # description = re.search('\)*,*.*', line).group(1)
                    # description = re.search('\),+.*', line).group(1)
                    # description = re.search('\]\s.*\)(.*)', line).group(1)
                    description = re.search('\]\](\)*,*.*)', line).group(1)
                except Exception as e:
                    print("#################################")
                    print("#### description parse error ####")
                    print(e)
                    print(index, line)
                    filepath.write(line + '\n')
                    print("#################################")
                # print(this_sec)
                # print(sub_sec)
                poets[sec].append({
                    'poet': poet,
                    'date': date,
                    'description': description
                })
        else:
            #print(sec)
            if re.match('===\w', line[0:4]):
                sub_sec = line.strip().replace("=", "")
                #print(sub_sec)
            if line.strip().replace("=", "") in poets[sec]:
                #print(sub_sec)
                this_sec[sub_sec] = []
            if sub_sec != '':
                if re.match('\*\[\[', line[0:3]):
                    try:
                        poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                        #print(poet)
                    except:
                        print("##########################")
                        print("#### poet parse error ####")
                        print(index, line)
                        filepath.write(line + '\n')
                        print(line[-2:])
                        print("##########################")
                    date_m = re.search('\((.*)\)', line)
                    try:
                        date = date_m.group(0)
                    except Exception as e:
                        print("##########################")
                        print("#### date parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        date = ""
                        print("##########################")
                    try:
                        # description = re.search('\)*,*.*', line).group(1)
                        # description = re.search('\),+.*', line).group(0)
                        # description = re.search('\]\s.*\)(.*)', line).group(0)
                        description = re.search('\]\](\)*,*.*)', line).group(1)

                    except Exception as e:
                        print("#################################")
                        print("#### description parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        description = ""
                        print("#################################")

                    this_sec[sub_sec].append({
                        'poet': poet,
                        'date': date,
                        'description': description
                    })


*[[John Adams (poet)|John Adams]] (1704–1740), American poet
##########################
#### date parse error ####
'NoneType' object has no attribute 'group'
163 *[[Renée Ashley]], American poet and novelist.
##########################
##########################
#### date parse error ####
'NoneType' object has no attribute 'group'
410 *[[Alison Calder]], Canadian poet and educator
##########################
##########################
#### date parse error ####
'NoneType' object has no attribute 'group'
581 *[[Thomas of Britain|Thomas d'Angleterre]], 12th-century poet writing in [[Old French]]
##########################
##########################
#### date parse error ####
'NoneType' object has no attribute 'group'
636 *[[Edward de Vere, 17th Earl of Oxford]], courtier and poet praised also for lost plays
##########################
##########################
#### date parse error ####
'NoneType' object has no attribute 'group'
660 *[[Gojko Đogo]], Serbian poet
##########################

In [12]:
# inspect final data element
# textDATA["parse"]["wikitext"]["*"].split('\n')[-11]

In [26]:
# inspect final poet dict
pprint(poets)

{'A': {'Ab–Ak': [{'date': '(born 1941)',
                  'description': ' (born 1941), American poet',
                  'poet': 'Jonathan Aaron'},
                 {'date': '(born 1966)',
                  'description': ' (born 1966), Nigerian poet',
                  'poet': 'Chris Abani'},
                 {'date': '(1842–1911)',
                  'description': ' (1842–1911), American poet',
                  'poet': 'Henry Abbey'},
                 {'date': '(1872–1958)',
                  'description': ' (1872–1958), American poet, novelist and '
                                 'short story writer',
                  'poet': 'Eleanor Hallowell Abbott'},
                 {'date': '(fl. 18th c.)',
                  'description': ' (fl. 18th c.), Welsh balladeer',
                  'poet': 'Siôn Abel'},
                 {'date': '(1881–1938)',
                  'description': ' (1881–1938), English poet and literary '
                                 'critic',
                

              {'date': '(poet)|James Berry]] (born 1924)',
               'description': ' (born 1924), Jamaican poet based in England',
               'poet': 'James Berry (poet)|James Berry'},
              {'date': '(born 1934)',
               'description': ' (born 1934), American man of letters, critic '
                              'and farmer',
               'poet': 'Wendell Berry'},
              {'date': '(1914–1972)',
               'description': ' (1914–1972), American poet and scholar',
               'poet': 'John Berryman'},
              {'date': '(1776–1836)',
               'description': ' (1776–1836), Hungarian poet',
               'poet': 'Dániel Berzsenyi'},
              {'date': '(1874–1945)',
               'description': ' (1874–1945), New Zealand poet and social '
                              'worker',
               'poet': 'Ursula Bethell|Mary Ursula Bethell'},
              {'date': '(1906–1984)',
               'description': ' (1906–1984), English p

                  'poet': 'Aimé Césaire'},
                 {'date': '(1923–2006)',
                  'description': ' (1923–2006), Portuguese [[surrealist]] poet',
                  'poet': 'Mário Cesariny de Vasconcelos'},
                 {'date': '(1832–1874)',
                  'description': ' (1832–1874), Cuban poet',
                  'poet': 'Úrsula Céspedes'},
                 {'date': '(born 1951)',
                  'description': ' (born 1951), Hindi author and poet',
                  'poet': 'Ashok Chakradhar'},
                 {'date': '(fl. 1600)',
                  'description': ' (fl. 1600), English poet',
                  'poet': 'John Chalkhill'},
                 {'date': '(1595–1674)',
                  'description': ' (1595–1674), French poet and critic in '
                                 '[[Louis XIV of France|Grand Siècle]]',
                  'poet': 'Jean Chapelain'},
                 {'date': '(poet)|Arthur Chapman]] (1873–1935)',
                  'd

        'poet': 'Jill Alexander Essbaum'},
       {'date': '(1889–1974)',
        'description': ' (1889–1974), Yiddish American poet and carpenter',
        'poet': 'Alter Esselin'},
       {'date': '(1935–2006)',
        'description': ' (1935–2006), French poet',
        'poet': 'Claude Esteban'},
       {'date': '(born 1963)',
        'description': ' (born 1963), American slam poet and musician',
        'poet': 'Maggie Estep'},
       {'date': '(1947–1993)',
        'description': ' (1947–1993), American poet and editor',
        'poet': 'Jerry Estrin'},
       {'date': '(480–406 BC)',
        'description': ' (480–406 BC), Athenian tragedian',
        'poet': 'Euripides'},
       {'date': '(1909–1958)',
        'description': ' (1909–1958), English poet and novelist',
        'poet': 'Margiad Evans'},
       {'date': '(born 1923)',
        'description': ' (born 1923), African-American poet',
        'poet': 'Mari Evans'},
       {'date': '(Brother Antoninus) (1912–1994)',
     

              {'date': '(born 1958)',
               'description': ' (born 1958), American poet, writer, and '
                              'performer',
               'poet': 'Allison Hedge Coke'},
              {'date': '(born 1959)',
               'description': ' (born 1959), Swiss writer and translator',
               'poet': 'Markus Hediger'},
              {'date': '(born 1953)',
               'description': ' (born 1953), English performance poet, '
                              'comedian, and songwriter',
               'poet': 'John Hegley'},
              {'date': '(1797–1856)',
               'description': ' (1797–1856), German poet, essayist, and '
                              'literary critic',
               'poet': 'Heinrich Heine'},
              {'date': '(born 1941)',
               'description': ' (born 1941), American poet, essayist, and '
                              'translator',
               'poet': 'Lyn Hejinian'},
              {'date': '(1089–1172)

               'poet': 'Letitia Elizabeth Landon'},
              {'date': '(1775–1864)',
               'description': ' (1775–1864), English writer and poet',
               'poet': 'Walter Savage Landor'},
              {'date': '(c. 1332 – c. 1386)',
               'description': ' (c. 1332 – c. 1386) probable English author of '
                              'dream-vision [[Piers Plowman]]',
               'poet': 'William Langland'},
              {'date': '(1569–1645)',
               'description': ' (1569–1645), English poet',
               'poet': 'Emilia Lanier'},
              {'date': '(c. 1510–1556)',
               'description': ' (c. 1510–1556), Hungarian poet and historian',
               'poet': 'Sebestyén Tinódi Lantos'},
              {'date': '(Lau-tzu) (fl. 6th c. BC)',
               'description': ' (Lau-tzu) (fl. 6th c. BC), Chinese philosopher '
                              'and poet of history of ancient China',
               'poet': 'Laozi'},
          

                 {'date': '(1837–1913)',
                  'description': ' (1837–1913), American poet',
                  'poet': 'Joaquin Miller'},
                 {'date': '(born 1956)',
                  'description': ' (born 1956), American poet',
                  'poet': 'Leslie Adrienne Miller'},
                 {'date': '(poet)|Thomas Miller]] (1807–1874)',
                  'description': ' (1807–1874), English poet',
                  'poet': 'Thomas Miller (poet)|Thomas Miller'},
                 {'date': '(1924–1998)',
                  'description': ' (1924–1998), American writer and poet',
                  'poet': 'Vassar Miller'},
                 {'date': '(1918–2002)',
                  'description': ' (1918–2002), Irish comedian, poet, and '
                                 'musician',
                  'poet': 'Spike Milligan'},
                 {'date': '(1911–2004)',
                  'description': ' (1911–2004), Polish poet; 1980 Nobel Prize '
            

                 {'date': '(poet)|Allan Ramsay]] (1686–1758)',
                  'description': ' (1686–1758), Scottish poet, playwright and '
                                 'publisher',
                  'poet': 'Allan Ramsay (poet)|Allan Ramsay'},
                 {'date': '(1914–2000)',
                  'description': ' (1914–2000), African-American poet and '
                                 'publisher',
                  'poet': 'Dudley Randall'},
                 {'date': '(poet)|Thomas Randolph]] (1605–1635)',
                  'description': ' (1605–1635), English poet and dramatist',
                  'poet': 'Thomas Randolph (poet)|Thomas Randolph'},
                 {'date': '(1888–1974)',
                  'description': ' (1888–1974), American poet, essayist, and '
                                 'magazine editor',
                  'poet': 'John Crowe Ransom'},
                 {'date': '(1936–2016)',
                  'description': ' (1936–2016), Telugu poet and lit

                                 "''[[The Faerie Queene]]''",
                  'poet': 'Edmund Spenser'}],
       'St–Sz': [{'date': '(1878–1957)',
                  'description': ' (1878–1957), Polish poet',
                  'poet': 'Leopold Staff'},
                 {'date': '(poet)|William Stafford]] (1914–1993)',
                  'description': ' (1914–1993), American poet and pacifist; US '
                                 'Poet Laureate 1970–71',
                  'poet': 'William Stafford (poet)|William Stafford'},
                 {'date': '(born 1968)',
                  'description': ' (born 1968), American poet and translator',
                  'poet': 'A.E. Stallings'},
                 {'date': '(born 1935)',
                  'description': ' (born 1935), English academic, poet and '
                                 'literary critic',
                  'poet': 'Jon Stallworthy'},
                 {'date': '(1919–1972)',
                  'description': ' (1919–1972)

                  'description': ' (born 1958), American poet; South Carolina '
                                 'Poet Laureate',
                  'poet': 'Marjory Heath Wentworth'},
                 {'date': '(1707–1788)',
                  'description': ' (1707–1788), English leader of Methodist '
                                 'movement, and prolific hymnist',
                  'poet': 'Charles Wesley'},
                 {'date': '(1703–1756)',
                  'description': ' (1703–1756), English poet, translator and '
                                 'Christian apologist',
                  'poet': 'Gilbert West'},
                 {'date': '(1923–2002)',
                  'description': ' (1923–2002), American poet, Zen Buddhist, '
                                 'and figure in [[San Francisco Renaissance]]',
                  'poet': 'Philip Whalen'},
                 {'date': '(1890–1945)',
                  'description': ' (1890–1945), Austrian-Bohemian novelist, '
   

In [14]:
# transfer poet dict to json file
import json
with open('poets.json', 'w') as filepath:
    json.dump(poets, filepath)

In [30]:
poet_names = []
with open('poet_names.txt', 'w+', encoding="utf-8") as fp:
    for k, v in poets.items():
        if type(v).__name__ == 'list':
            for poet in v:
                poet_names.append(poet['poet'])
                fp.write(poet['poet'] + '\n')
        else:
            for sub_sec, _poets in v.items():
                for poet in _poets:
                    poet_names.append(poet['poet'])
                    fp.write(poet['poet'] + '\n')

{'poet': 'Jonathan Aaron', 'date': '(born 1941)', 'description': ' (born 1941), American poet'}
{'poet': 'Chris Abani', 'date': '(born 1966)', 'description': ' (born 1966), Nigerian poet'}
{'poet': 'Henry Abbey', 'date': '(1842–1911)', 'description': ' (1842–1911), American poet'}
{'poet': 'Eleanor Hallowell Abbott', 'date': '(1872–1958)', 'description': ' (1872–1958), American poet, novelist and short story writer'}
{'poet': 'Siôn Abel', 'date': '(fl. 18th c.)', 'description': ' (fl. 18th c.), Welsh balladeer'}
{'poet': 'Lascelles Abercrombie', 'date': '(1881–1938)', 'description': ' (1881–1938), English poet and literary critic'}
{'poet': 'Arthur Talmage Abernethy', 'date': '(1872–1956)', 'description': ' (1872–1956), American journalist, minister, scholar; first [[North Carolina Poet Laureate]]'}
{'poet': 'Sam Abrams', 'date': '(born 1935)', 'description': ' (born 1935), American poet'}
{'poet': 'Seth Abramson', 'date': '(born 1976)', 'description': ' (born 1976), American poet, edi

In [ ]:
# get poet links
PARAMS = {
    "action": "parse",
    "pageid": "64618",
    "format": "json",
    "prop": "links"
}

linkR = S.get(url=URL, params=PARAMS)
linkDATA = linkR.json()

#pprint(linkDATA["parse"]["links"])

In [ ]:
# getting types
# for key, value in poets.items():
    # print(value.__class__.__name__)
    # print(type(value).__name__)
    

In [ ]:
# test to see how to parse sections
for sec in sections:
    print("##" + sec + "##")
    
    for line in textDATA["parse"]["wikitext"]["*"].split('\n'):
        if line.replace(" ", "") == ("==" + sec + "=="):
            print(line)
        if line.replace(" ", "") == ("===" + sec + "==="):
            print(line)
            

In [ ]:
# whole page sample
petPARAMS = {
    "action": "parse",
    "page": "Pet door",
    "format": "json"
}

petR = S.get(url=URL, params=petPARAMS)
petDATA = petR.json()

# print(petDATA["parse"]["text"]["*"])

In [ ]:
display = ''
for i in range(100):
    if i%15 == 0:
        print(display)
        display = 'FizzBuzz'
        print(display)
        continue
    if i%5 == 0:
        print(display)
        display = 'Buzz'
        print(display)
        continue
    if i%3 == 0:
        print(display)
        display = 'Fizz'
        print(display)
        continue
    else:
        print(i)
        continue